In [2]:
import numpy as np
import random
import gensim
import pandas as pd
from owlready2 import *

Load ontology and embeddings

In [3]:
#OntoGeoLógica populada (OntoGeoLogicaPopulada.owl)
onto = get_ontology("../../KnowledgeGraph/OntoGeoLogicaInstanciasRelacoes.owl")
onto.load()

get_ontology("http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#")

In [4]:
# Modelo OWL2Vec - 
#PetroOntoVec = gensim.models.Word2Vec.load("../../Embeddings/PetroOntoVec/Base/outputontology.embeddings")
PetroOntoVec = gensim.models.Word2Vec.load("../../Embeddings/PetroOntoVec/Petrovec-OeG_NP2/outputontology.embeddings")


### Carregando dataset

In [5]:
# Listas de treino
text_treino = np.load('../PetroNER-LinkedEntity/treino - sentences.npy')
entities_treino = np.load('../PetroNER-LinkedEntity/treino - entities.npy')
classes_treino = np.load('../PetroNER-LinkedEntity/treino - classes.npy')
URI_treino = np.load('../PetroNER-LinkedEntity/treino - URI.npy', allow_pickle=True)
#URIvec_treino = np.load('../../Corpora/PetroNER-LinkedEntity/treino - URI_vectors.npy')

# Listas de validação
text_valid = np.load('../PetroNER-LinkedEntity/valid - sentences.npy')
entities_valid = np.load('../PetroNER-LinkedEntity/valid - entities.npy')
classes_valid = np.load('../PetroNER-LinkedEntity/valid - classes.npy')
URI_valid = np.load('../PetroNER-LinkedEntity/valid - URI.npy', allow_pickle=True)
#URIvec_valid = np.load('../../Corpora/PetroNER-LinkedEntity/valid - URI_vectors.npy')

# Listas de teste
text_teste = np.load('../PetroNER-LinkedEntity/teste - sentences.npy')
entities_teste = np.load('../PetroNER-LinkedEntity/teste - entities.npy')
classes_teste = np.load('../PetroNER-LinkedEntity/teste - classes.npy')
URI_teste = np.load('../PetroNER-LinkedEntity/teste - URI.npy', allow_pickle=True)
#URIvec_teste = np.load('../../Corpora/PetroNER-LinkedEntity/teste - URI_vectors.npy')

In [6]:
print ('Treino')
print('Texto: ', len(text_treino))
print('Entidades: ', len(entities_treino))
print('Classes: ', len(classes_treino))
print('URI: ', len(URI_treino))
#print('URIvec: :', len(URIvec_treino))

print ('\n Validação')
print('Texto: ', len(text_valid))
print('Entidades: ', len(entities_valid))
print('Classes: ', len(classes_valid))
print('URI: ', len(URI_valid))
#print('URIvec: :', len(URIvec_valid))

print ('\n Teste')
print('Texto: ', len(text_teste))
print('Entidades: ', len(entities_teste))
print('Classes: ', len(classes_teste))
print('URI: ', len(URI_teste))
#print('URIvec: :', len(URIvec_teste))

Treino
Texto:  14171
Entidades:  14171
Classes:  14171
URI:  14171

 Validação
Texto:  1775
Entidades:  1775
Classes:  1775
URI:  1775

 Teste
Texto:  3191
Entidades:  3191
Classes:  3191
URI:  3191


### Buscando vetores do PetroOntoVec para cada URI

In [7]:
def buscando_URI_vec(text, entities, classes_NER, ENT_URI):

    # Separando as listas de sentenças, entidades, classes, URI e vetores
    text_new = []
    entities_new = []
    classes_NER_new = [] 
    ENT_URI_new = []
    URI_Vec_new = []

    URI_ausente_Onto = []

    n = 0 
    #m = 0
    #Iterando por todas as sentenças, desconsiderando aquelas cujas entidades não possuem URI
    for i in range(len(text)):
        n = n + 1
        if ENT_URI[i] != None:
            # Quebrando as entidades que possuem mais de um URI (ex: "#Aptian,#Miocene" e "#Cenozoic,#Mesozoic")
            ENT_URI_n = ENT_URI[i].split(',#')

            # Iterando pelas ENT_URI após serem quebradas e acrescentando '#" naquelas que perderam na operação de split 
            for ent in ENT_URI_n:
                if ent[0] != '#':
                    ent = '#' + ent
                try:
                    #Acrescentar a URI da entidade e o vetor da URI nas listas
                    URI_Vec_new.append(PetroOntoVec['http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2' + ent])
                    ENT_URI_new.append(ent)

                    # O texto, a entidade no texto e a classe são repetidas quando existem multiplas URI para uma mesma entidade
                    text_new.append(text[i])
                    entities_new.append(entities[i])
                    classes_NER_new.append(classes_NER[i])
                except:
                    #print("URI presente no arquivo CONLLU mas ausente na Ontologia: ", ent)
                    URI_ausente_Onto.append(ent)

    # Transformando as listas em Numpy array
    text = np.array(text_new)
    entities = np.array(entities_new)
    classes_NER = np.array(classes_NER_new) 
    ENT_URI = np.array(ENT_URI_new)
    URI_Vec = np.array(URI_Vec_new)
    URI_ausente_Onto = np.array(URI_ausente_Onto)

    print('Total de sentenças anotadas: ', n)
    print('N° de sentenças: ', len(text))
    print ('N° de entidades: ', len(entities))
    print('N° de classes: ', len(classes_NER))
    print('N° de entidades_ID: ', len(ENT_URI))       
    print('N° de vetores de URI: ', len(URI_Vec))       
    
    return(text, entities, classes_NER, ENT_URI, URI_Vec, URI_ausente_Onto)

In [8]:
#Treino
(text_treino,
 entities_treino,
 classes_treino,
 URI_treino,
 URIvec_treino,
 URI_ausente_Onto_treino) = buscando_URI_vec(text_treino,
                                             entities_treino,
                                             classes_treino,
                                             URI_treino)

#Validação
(text_valid,
 entities_valid,
 classes_valid,
 URI_valid,
 URIvec_valid,
 URI_ausente_Onto_valid) = buscando_URI_vec(text_valid,
                                            entities_valid,
                                            classes_valid,
                                            URI_valid)

#Teste
(text_teste,
 entities_teste,
 classes_teste,
 URI_teste,
 URIvec_teste,
 URI_ausente_Onto_teste) = buscando_URI_vec(text_teste,
                                            entities_teste,
                                            classes_teste,
                                            URI_teste)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Total de sentenças anotadas:  14171
N° de sentenças:  13465
N° de entidades:  13465
N° de classes:  13465
N° de entidades_ID:  13465
N° de vetores de URI:  13465
Total de sentenças anotadas:  1775
N° de sentenças:  1690
N° de entidades:  1690
N° de classes:  1690
N° de entidades_ID:  1690
N° de vetores de URI:  1690
Total de sentenças anotadas:  3191
N° de sentenças:  2997
N° de entidades:  2997
N° de classes:  2997
N° de entidades_ID:  2997
N° de vetores de URI:  2997


In [9]:
print('URI no PetroNER_Treino ausentes da ontologia: ')
print(set(URI_ausente_Onto_treino))
print('')
print('URI no PetroNER_Validação ausentes da ontologia: ')
print(set(URI_ausente_Onto_valid))
print('')
print('URI no PetroNER_Teste ausentes da ontologia: ')
print(set(URI_ausente_Onto_teste))

URI no PetroNER_Treino ausentes da ontologia: 
{'#inter_formacao_020', '#Rio_da_serra_Age', '#explotatory_well', '#INTER_BACIA_020', '#externo_textura_006', '#inter_grupo_002', '#porosidade_22', '#externo_textura_008', '#INTER_BACIA_008', '#EXTERNO_EST_FISICA_ROCHA_003', '#formation', '#appraisal_well', '#INTER_BACIA_021', '#INTER_BACIA_014', '#inter_membro_013', '#stratigraphic_well', '#special_well', '#EXTERNO_EST_FISICA_ROCHA_005', '#externo_formacao_003', '#externo_grupo_003', '#externo_formacao_004', '#porosidade_05', '#porosidade_24', '#INTER_BACIA_019', '#externo_grupo_005', '#INTER_BACIA_003', '#INTER_BACIA_015', '#INTER_BACIA_013', '#EXTERNO_EST_FISICA_ROCHA_006', '#inter_formacao_027', '#CAMP_CD_CAMPO_0817', '#inter_membro_011', '#inter_formacao_021', '#discovery_well', '#INTER_BACIA_005', '#grupo_05', '#inter_formacao_019', '#externo_textura_002', '#formation_fluid', '#generic_anthropogenic_fluid', '#dry_hole', '#inter_formacao_023', '#externo_membro_001', '#INTER_BACIA_011'

### Preparando dataset para treinar rede Siamesa

In [10]:
#Dataframe com as URI, sentenças e classes

def dataset_siamesa(classes, URI, URIvec, text, x_easy, x_hard):
       
    df = pd.DataFrame({'Classe': classes, 'URI': URI, 'Sentença': text}) #, 'URIvec': URIvec})
    
    # Arrays para armazenar as sentenças âncoras, positivas e negativas 
    URI_anchor = np.array([])
    URIvec_anchor = np.array([])
    dataset_anchor = np.array([])
    dataset_positive = np.array([])
    dataset_negative = np.array([])

    #Quantidade de exemplos negativos fáceis e difíceis
    #x_easy = 20
    #x_hard = 30

    # iterando por todas as URI
    for i in set(URI):

        try:
            
            # Subgrupo apenas com a URI selecionada
            df_URI = df[df['URI'] == i]

            # Classe da URI selecionada
            classe_da_URI = df_URI['Classe'].values[0]
            # Vetor da URI selecionada
            vetor_da_URI = URIvec[df_URI.index[0]]
            # Subgrupo com sentenças negativa 'fácil' (URI diferente e classe diferente).
            df_negative_easy = df[(df['URI'] != i) & (df['Classe'] != classe_da_URI)]
            # Subgrupo com sentenças negativa 'difícil' (URI diferente mas da mesma classe).
            df_negative_hard = df[(df['URI'] != i) & (df['Classe'] == classe_da_URI)]

            # Pular os casos onde só existe uma sentença para a URI selecionada
            if (len(df_URI) > 1) & (len(df_negative_easy) > 1) & (len(df_negative_hard) > 1):

                ### Construindo triplets com exemplos negativos fáceis

                # Selecionando as sentenças âncoras 
                # (verificamos o número de combinações possíveis entre sentenças com mesma URI "len(df_URI)*(len(df_URI) - 1)".
                #  Se X_easy for maior que esse valor, amostramos apenas essa quantidade de amostras.
                df_ancora = df_URI.sample(min(len(df_URI)*(len(df_URI) - 1) , x_easy), replace=True)

                # itera pelas senteças âncoras
                for row in df_ancora.iterrows():

                    anchor = row[1]['Sentença']
                    # Seleciona uma exemplo positivo diferente da sentença âncora
                    positive = df_URI[df_URI['Sentença'] != anchor].sample(1)['Sentença'].values[0]

                    # Seleciona uma sentença negativa
                    negative_easy = df_negative_easy.sample(1)['Sentença'].values[0]

                    # Concatena com as arrays que armazenam as senteças e URI
                    URI_anchor = np.concatenate((URI_anchor, np.array([np.str_(i)])), axis=0)
                    URIvec_anchor = np.concatenate((URIvec_anchor, vetor_da_URI), axis=0)
                    dataset_anchor = np.concatenate((dataset_anchor, np.array([anchor])), axis=0)
                    #dataset_anchor  = np.concatenate((dataset_anchor, anchor), axis=0)
                    dataset_positive = np.concatenate((dataset_positive, np.array([positive])), axis=0)
                    dataset_negative = np.concatenate((dataset_negative, np.array([negative_easy])), axis=0)

                ### Construindo triplets com exemplos negativos difíceis

                # Selecionando as sentenças âncoras 
                # (verificamos o número de combinações possíveis entre sentenças com mesma URI "len(df_URI)*(len(df_URI) - 1)".
                #  Se X_hard for maior que esse valor, amostramos apenas essa quantidade de amostras.
                
                df_ancora = df_URI.sample(min(len(df_URI)*(len(df_URI) - 1), x_hard), replace=True)

                # itera pelas senteças âncoras
                for row in df_ancora.iterrows():
                    anchor = row[1]['Sentença']

                    # Seleciona uma exemplo positivo diferente da sentença âncora
                    positive = df_URI[df_URI['Sentença'] != anchor].sample(1)['Sentença'].values[0]

                    # Seleciona uma sentença negativa
                    negative_hard = df_negative_hard.sample(1)['Sentença'].values[0]

                    # Concatena com as arrays que armazenam as senteças
                    URI_anchor = np.concatenate((URI_anchor, np.array([np.str_(i)])), axis=1)
                    URIvec_anchor = np.concatenate((URIvec_anchor, vetor_da_URI), axis=0)
                    dataset_anchor  = np.concatenate((dataset_anchor, np.array([anchor])), axis=0)
                    #dataset_anchor  = np.concatenate((dataset_anchor, anchor), axis=0)
                    dataset_positive  = np.concatenate((dataset_positive, np.array([positive])), axis=0)
                    dataset_negative  = np.concatenate((dataset_negative, np.array([negative_hard])), axis=0)

        except:
            pass
    
    #Ajustando o formato da array URIvec_anchor
    URIvec_anchor = np.reshape(URIvec_anchor, (-1, len(URIvec[0])))
    
    return(URI_anchor, URIvec_anchor, dataset_anchor, dataset_positive, dataset_negative)

In [11]:
(URI_anchor_treino,
 URIvec_anchor_treino,
 dataset_anchor_treino,
 dataset_positive_treino,
 dataset_negative_treino) = dataset_siamesa(classes_treino,
                                            URI_treino,
                                            URIvec_treino,
                                            text_treino, 200, 200)

(URI_anchor_valid, 
 URIvec_anchor_valid,
 dataset_anchor_valid,
 dataset_positive_valid,
 dataset_negative_valid) = dataset_siamesa(classes_valid,
                                           URI_valid,
                                           URIvec_valid,
                                           text_valid, 200, 200)

(URI_anchor_teste,
 URIvec_anchor_teste,
 dataset_anchor_teste,
 dataset_positive_teste,
 dataset_negative_teste) = dataset_siamesa(classes_teste,
                                           URI_teste,
                                           URIvec_teste,   
                                           text_teste, 200, 200)

In [12]:
print('Dataset treino: ', len(URI_anchor_treino))
print('Dataset validação: ', len(dataset_anchor_valid))
print('Dataset teste: ', len(dataset_anchor_teste))

Dataset treino:  34826
Dataset validação:  9368
Dataset teste:  13686


In [12]:
Dataset treino:  12448
Dataset validação:  3282
Dataset teste:  4948

SyntaxError: invalid syntax (<ipython-input-12-2214c4b690cb>, line 1)

In [13]:
n = 51

print(dataset_anchor_treino[n])
print(dataset_positive_treino[n])
print(dataset_negative_treino[n])
print(URI_anchor_treino[n])
print(URIvec_anchor_treino[n])

[ UNIDADE_CRONO ] Paleoceno | Nota-se, também, a ocorrência de grandes discordâncias angulares na base da Formação Calumbi( Santoniano/Turoniano), e próximas ao topo do [ E ] Paleoceno [ / E ]..
[ UNIDADE_CRONO ] Paleoceno | Apesar disso, na fase inícial da progradação, as paleobatimetrias aumentam até o [ E ] Paleoceno [ / E ], como resultado de uma subsidência que ainda continua maior do que o aporte sedimentar( fase final da subsidência térmica”..
[ ROCHA ] arenito | Becker et al.( 1989) identificaram seis litofácies: conglomerado arenoso( Ca), [ E ] arenito [ / E ] conglomerático( Ac), arenito maciço( Am), interlaminado( 1), arenito estratificado( Ae) e arenito argiloso deformado( D)..
#Paleocene
[-0.28884128  0.29546309 -0.17015468  0.20688121 -0.34036124 -0.72685421
 -1.48428321 -0.85393631  0.03652421  0.46401012 -0.68416041 -0.768749
 -0.03291129  1.92501605  0.20827916 -0.10501655 -0.2687076   0.08026255
  0.97931963 -0.37341329  0.43900031  0.22620183 -0.51032269  1.3114686
 

### Salvando dataset


In [14]:
#Treino
with open('URI_anchor_treino.npy', 'wb') as f:
    np.save(f, URI_anchor_treino)
with open('URIvec_anchor_treino.npy', 'wb') as f:
    np.save(f, URIvec_anchor_treino)
with open('anchor_treino.npy', 'wb') as f:
    np.save(f, dataset_anchor_treino)
with open('positive_treino.npy', 'wb') as f:
    np.save(f, dataset_positive_treino)
with open('negative_treino.npy', 'wb') as f:
    np.save(f, dataset_negative_treino)

    
#Validação
with open('URI_anchor_valid.npy', 'wb') as f:
    np.save(f, URI_anchor_valid)
with open('URIvec_anchor_valid.npy', 'wb') as f:
    np.save(f, URIvec_anchor_valid)
with open('anchor_valid.npy', 'wb') as f:
    np.save(f, dataset_anchor_valid)
with open('positive_valid.npy', 'wb') as f:
    np.save(f, dataset_positive_valid)
with open('negative_valid.npy', 'wb') as f:
    np.save(f, dataset_negative_valid)
    
#Teste
with open('URI_anchor_teste.npy', 'wb') as f:
    np.save(f, URI_anchor_teste)
with open('URIvec_anchor_teste.npy', 'wb') as f:
    np.save(f, URIvec_anchor_teste)
with open('anchor_teste.npy', 'wb') as f:
    np.save(f, dataset_anchor_teste)
with open('positive_teste.npy', 'wb') as f:
    np.save(f, dataset_positive_teste)
with open('negative_teste.npy', 'wb') as f:
    np.save(f, dataset_negative_teste)